# Attention 的 四种写法

In [5]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

d_model = 512
num_heads = 8
hidden_dim = 256
dropout = 0.2

ModuleNotFoundError: No module named 'torch'

In [2]:
!pip install torch

## 原始

In [ ]:
class Self_Attention_V1(nn.Module):
    def __init__(self, hiddin_dim):
        super().__init__()
        self.hiddin_dim = hiddin_dim
        self.query_proj = nn.Linear(hiddin_dim, hiddin_dim)
        self.key_proj = nn.Linear(hiddin_dim, hiddin_dim)
        self.value_proj = nn.Linear(hiddin_dim, hiddin_dim)



    # X [batch_size,seq,embedding_dim]
    def forward(self, X):
        Q = self.query_proj(X)
        K = self.key_proj(X)
        V = self.value_proj(X)
        attention_weight= F.softmax(Q@K.permute(0,2,1)/math.sqrt(self.hiddin_dim), dim=-1)
        print(attention_weight)
        return (
           attention_weight @V
        )

In [ ]:
x1 =torch.randn(3, 3, d_model)
x2 = torch.randn(3, 3, d_model)

In [ ]:
x1

In [ ]:
x2

In [ ]:
self_attention_v1 = Self_Attention_V1(d_model)

In [ ]:
self_attention_v1(x1)

## QKV 转换为Matrix

In [ ]:
class Self_Attention_V2(nn.Module):
    def __init__(self, hiddin_dim):
        super().__init__()
        self.hiddin_dim = hiddin_dim
        self.QKV_proj = nn.Linear(hiddin_dim, hiddin_dim*3)
    def forward(self, X):
        QKV = self.QKV_proj(X)
        Q,K,V = torch.split(QKV,hidden_dim,dim=-1)
        attention_weight= F.softmax(Q@K.transpose(-2,-1)/math.sqrt(hidden_dim),dim=-1)
        return attention_weight@V


In [ ]:
self_attention_v2= Self_Attention_V2(d_model)


In [ ]:
self_attention_v2(x1)

## 加细节写法

In [ ]:
# drop out
# attention mask
# output matrix

class Self_Attention_V3(nn.Module):
    def __init__(self, hiddin_dim,dropout=0.2):
        super().__init__()
        self.hiddin_dim = hiddin_dim
        self.QKV_proj = nn.Linear(hiddin_dim, hiddin_dim*3)
        self.dropout = nn.Dropout(dropout)

    def forward(self, X,mask=None):
        QKV = self.QKV_proj(X)
        Q,K,V= torch.split(QKV,hidden_dim,dim=-1)
        attention_weight= F.softmax(Q@K.transpose(-2,-1)/math.sqrt(hidden_dim),dim=-1)
        if mask is not None:
            attention_weight = attention_weight.masked_fill(mask==0,float('-inf'))
        attention_weight = self.dropout(attention_weight)
        return attention_weight@V

## Interview写法(装逼写法）

In [ ]:
class Self_Attention_V4(nn.Module):
    def __init__(self, hiddin_dim: int,dropout=0.2)->None:
        super().__init__()
        self.hiddin_dim = hiddin_dim
        self.query_proj = nn.Linear(hiddin_dim, hiddin_dim)
        self.key_proj = nn.Linear(hiddin_dim, hiddin_dim)
        self.value_proj = nn.Linear(hiddin_dim, hiddin_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, X,mask=None):
        Q = self.query_proj(X)
        K = self.key_proj(X)
        V = self.value_proj(X)
        attention_weight= Q@K.transpose(-2,-1)/math.sqrt(hidden_dim)
        if mask is not None:
            attention_weight=attention_weight.masked_fill(mask==0,float('-inf'))
        attention_weight = F.softmax(attention_weight,dim=-1)
        attention_weight = self.dropout(attention_weight)
        return attention_weight@V
